In [65]:
from crypt import methods
from logging.handlers import BaseRotatingHandler
from pyexpat import ErrorString
from tkinter import N
from tokenize import PlainToken
from ciao_contrib.runtool import *
import os
import glob
import numpy as np
import sys
from re import sub
import subprocess
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
import numpy as np
import random
from scipy.stats import entropy
from astropy.stats import bayesian_blocks
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from matplotlib import axes
from astropy.stats import histogram
from statistics import mean, median, median_high
from scipy.stats import norm
from scipy import interpolate
from numpy import trapz
import astropy.stats as astats
from region import *

BEHR_DIR = '/Users/steven/BEHR/BEHR/'
DATA_DIR = '/Users/steven/Desktop/CSCData/23022need/'
evt_file = glob.glob(f'{DATA_DIR}*evt6*')[0]
regevt_file = glob.glob(f'{DATA_DIR}*evt6_filtered*')[0]
src_region = glob.glob(f'{DATA_DIR}*reg6*')[0]
bkg_region = glob.glob(f'{DATA_DIR}*bkg6*')[0]
bkgevt_file = glob.glob(f'{DATA_DIR}*evt6_back*')[0]

# filtered_filename = evt_file.replace(".fits", "_background.fits")

# dmcopy(f'{evt_file}[sky=region({bkg_region})]', filtered_filename)

a=1

print(evt_file )
print(src_region)
print(bkg_region)

src_a = CXCRegion(f'region({src_region})')
bkg_a = CXCRegion(f'region({bkg_region})')
print(src_a.area())
print(bkg_a.area())
a_ratio = bkg_a.area()/src_a.area()
print(a_ratio)


def behr_hr(area,conf = '68.00'):
    BEHR_DIR = '/Users/steven/BBBB/BEHR/'
    DATA_DIR = '/Users/steven/Desktop/CSCData/23022need/'
    evt_file = glob.glob(f'{DATA_DIR}*evt6*')[0]
    regevt = glob.glob(f'{DATA_DIR}*evt6_filtered*')[0]
    src_region = glob.glob(f'{DATA_DIR}*reg6*')[0]
    bkg_region = glob.glob(f'{DATA_DIR}*bkg5*')[0]
    bevt = glob.glob(f'{DATA_DIR}*evt6_back*')[0]

    with fits.open(regevt) as hdul:
        # Events dataframe
        events = hdul["Events"].data
        events_table = Table(events)
        events_cols = events.columns.names
        df_events = pd.DataFrame.from_records(events_table, columns=events_cols)
        df_events = df_events.sort_values(by=["time"])
        # GTI (Good Time Interval) dataframe
        gti = hdul["GTI"].data
        gti_table = Table(gti)
        gti_cols = gti.columns.names
        df_gti = pd.DataFrame.from_records(gti_table, columns=gti_cols)
        # Apply GTI filter to events
        gti_mask = np.zeros(len(df_events), dtype=bool)
        for i in range(len(df_gti)):
            start = df_gti.iloc[i]['START']
            stop = df_gti.iloc[i]['STOP']
            gti_mask |= (df_events["time"] >= start) & (df_events["time"] < stop)
        df_events = df_events[gti_mask]
        # Apply energy, pha, grade filters to events
        df_events = df_events[(df_events['pha']>40) & (df_events['grade']>=0) & (df_events['energy']>500) & (df_events['energy']<7000)]
        start_time = min(df_events.time.values)
        src_times = df_events.time.values - start_time
        src_energies = df_events.energy.values

    with fits.open(bevt) as hdul:
        # Events dataframe
        bevents = hdul["Events"].data
        bevents_table = Table(bevents)
        bevents_cols = bevents.columns.names
        df_bevents = pd.DataFrame.from_records(bevents_table, columns=bevents_cols)
        df_bevents = df_bevents.sort_values(by=["time"])
        # GTI (Good Time Interval) dataframe
        bgti = hdul["GTI"].data
        bgti_table = Table(bgti)
        bgti_cols = bgti.columns.names
        df_bgti = pd.DataFrame.from_records(bgti_table, columns=bgti_cols)
        # Apply GTI filter to events
        bgti_mask = np.zeros(len(df_bevents), dtype=bool)
        for i in range(len(df_bgti)):
            start = df_bgti.iloc[i]['START']
            stop = df_bgti.iloc[i]['STOP']
            bgti_mask |= (df_bevents["time"] >= start) & (df_bevents["time"] < stop)
        df_bevents = df_bevents[bgti_mask]
        # Apply energy, pha, grade filters to events
        df_bevents = df_bevents[(df_bevents['pha']>40) & (df_bevents['grade']>=0) & (df_bevents['energy']>500) & (df_bevents['energy']<7000)]
        bkg_times = df_bevents.time.values - start_time
        bkg_energies = df_bevents.energy.values

    bb_bins = astats.bayesian_blocks(src_times, fitness='events',p0 = 0.01) # p0 = 0.1
    bin_widths = bb_bins[1:] - bb_bins[:-1]
    counts, _ = np.histogram(src_times, bins=bb_bins)

    times_and_energies = np.column_stack((src_times,src_energies))
    bkg_te = np.column_stack((bkg_times,bkg_energies))
    # define soft/medium/hard bands
    soft_filter = times_and_energies[:,1] < 1200
    medium_filter = (times_and_energies[:,1] < 2000) *(times_and_energies[:,1] > 1200)
    hard_filter = times_and_energies[:,1] > 2000
    soft_medium_filter = times_and_energies[:,1] < 2000
    soft_filter_b = bkg_te[:,1] < 1200
    medium_filter_b = (bkg_te[:,1] < 2000) *(bkg_te[:,1] > 1200)
    hard_filter_b = bkg_te[:,1] > 2000
    soft_medium_filter_b = bkg_te[:,1] < 2000
    # generate data for each band
    soft_lc = times_and_energies[soft_filter]
    medium_lc = times_and_energies[medium_filter]
    hard_lc =times_and_energies[hard_filter]
    sm_lc = times_and_energies[soft_medium_filter]
    soft_b = bkg_te[soft_filter_b]
    medium_b = bkg_te[medium_filter_b]
    hard_b = bkg_te[hard_filter_b]
    sm_b = bkg_te[soft_medium_filter_b]
    #counts
    counts_s, _ = np.histogram(soft_lc[:,0], bins=   bb_bins)
    counts_m, _ = np.histogram(medium_lc[:,0], bins=   bb_bins)
    counts_h, _ = np.histogram(hard_lc[:,0], bins=   bb_bins)
    counts_sm, _ = np.histogram(sm_lc[:,0], bins=   bb_bins)
    counts_s_b, _ = np.histogram(soft_b[:,0], bins=   bb_bins)
    counts_m_b, _ = np.histogram(medium_b[:,0], bins=   bb_bins)
    counts_h_b, _ = np.histogram(hard_b[:,0], bins=   bb_bins)
    counts_sm_b, _ = np.histogram(sm_b[:,0], bins=   bb_bins)

    outfile = f'{DATA_DIR}/behr' 
    for i in range(len(counts_s)):
        with open(outfile,'w') as writeto:
            writeto.write(f'cd {BEHR_DIR}')
            writeto.write(f'\n echo "softsrc={counts_sm[i]} hardsrc={counts_h[i]}   softbkg={counts_sm_b[i]}   hardbkg={counts_h_b[i]} softarea={area} outputPr=True algo=quad"')
            writeto.write(f'\n./BEHR softsrc={counts_s[i]} hardsrc={counts_h[i]}   softbkg={counts_sm_b[i]}   hardbkg={counts_h_b[i]}  softarea={area} output={BEHR_DIR}/{i}_BEHRresults level={conf}')

    subprocess.run(f'bash {outfile}', shell = True)


    return counts_sm, counts_sm_b

# sm, smb = behr_hr(20,conf = '68.00')


# def sig_make_behr_running_avg(evt,srcreg,bkgreg,BEHR_DIR,outfile,BEHR_outdir,N,confidence='68.00'):
#     soft_src,hard_src,soft_bkg,hard_bkg = constant_count_split(evt,srcreg,bkgreg,2000,N)

#     hard_area = region_area(evt,bkgreg,3000)/region_area(evt,srcreg,3000)
#     soft_area = region_area(evt,bkgreg,1000)/region_area(evt,srcreg,1000)
    
#     ## need list of 2N+1 
    

#     with open(outfile,'w') as writeto:
#         writeto.write(f'cd {BEHR_DIR}')
#         for i in range(len(soft_src)):
#             writeto.write(f'\n echo "softsrc={soft_src[i]} hardsrc={hard_src[i]}   softbkg={soft_bkg[i]}   hardbkg={hard_bkg[i]}"')
#             writeto.write(f'\n./BEHR softsrc={soft_src[i]} hardsrc={hard_src[i]}   softbkg={soft_bkg[i]}   hardbkg={hard_bkg[i]}   softarea={soft_area} hardarea={hard_area} output={BEHR_outdir}/{i}_BEHRresults level={confidence}')


#     def sig_make_behr_running_avg(evt,srcreg,bkgreg,BEHR_DIR,outfile,BEHR_outdir,N,confidence='68.00'):

#     soft_src,hard_src,soft_bkg,hard_bkg = constant_count_split(evt,srcreg,bkgreg,2000,N)

#     hard_area = region_area(evt,bkgreg,3000)/region_area(evt,srcreg,3000)
#     soft_area = region_area(evt,bkgreg,1000)/region_area(evt,srcreg,1000)
    
#     ## need list of 2N+1 
    

#     with open(outfile,'w') as writeto:
#         writeto.write(f'cd {BEHR_DIR}')
#         for i in range(len(soft_src)):
#             writeto.write(f'\n echo "softsrc={soft_src[i]} hardsrc={hard_src[i]}   softbkg={soft_bkg[i]}   hardbkg={hard_bkg[i]}"')
#             writeto.write(f'\n./BEHR softsrc={soft_src[i]} hardsrc={hard_src[i]}   softbkg={soft_bkg[i]}   hardbkg={hard_bkg[i]}   softarea={soft_area} hardarea={hard_area} output={BEHR_outdir}/{i}_BEHRresults level={confidence}')

    

# def sig_run_BEHR(bash_file):
#     subprocess.run(f'bash {bash_file}', shell = True)


    # bb_s = astats.bayesian_blocks(soft_lc[:,0], fitness='events',p0 = 0.01) # p0 = 0.
    # counts_s, _ = np.histogram(soft_lc[:,0], bins=   bb_bins)
    # bb_m = astats.bayesian_blocks(medium_lc[:,0], fitness='events',p0 = 0.01) # p0 = 0.
    # counts_m, _ = np.histogram(medium_lc[:,0], bins=   bb_bins)
    # bb_h = astats.bayesian_blocks(hard_lc[:,0], fitness='events',p0 = 0.01) # p0 = 0.
    # counts_h, _ = np.histogram(hard_lc[:,0], bins=   bb_bins)
    # bb_sm = astats.bayesian_blocks(sm_lc[:,0], fitness='events',p0 = 0.01) # p0 = 0.
    # counts_sm, _ = np.histogram(sm_lc[:,0], bins=  bb_bins)
    # # # union them, and remove outliers
    # edges_union = np.concatenate((bb_s,bb_m,bb_h))
    # sorted_edges_union = np.sort(edges_union)

    # fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    # ax.fill_between(bb_bins, np.append(counts, counts[-1]), step="post", color = 'red', alpha=0.4, label='Bayesian Blocks Counts')
    # ax.tick_params(direction='in', top=True)
    # ax.minorticks_on()
    # ax.set_ylim([0,max(counts)*1.1])
    # ax.yaxis.label.set_color('red')
    # ax.tick_params(axis='y', colors='red')
    # ax.set_ylabel('Counts')
    # countrate = counts/bin_widths 
    # bin_centers = (bb_bins[:-1] + bb_bins[1:]) / 2
    # ax2 = ax.twinx()
    # rate = ax2.step(bb_bins, np.append(countrate, countrate[-1]), where='post', color='blue',label = 'Bayesian Blocks Count Rate')
    # ax2.minorticks_on()
    # ax2.set_ylabel('Count Rate [1/s]')
    # ax2.yaxis.label.set_color('blue')
    # ax2.tick_params(axis='y', colors='blue')
    # ax2.set_ylim([0,max(countrate)*1.1])
    # ax2.tick_params(direction='in', top=True)
    # # Combine all labels in one legend
    # handles, labels = ax.get_legend_handles_labels()
    # handles += rate
    # labels += ['Bayesian Blocks Count Rate']
    # plt.legend(handles=handles, labels=labels, loc='upper left', frameon=False, fontsize = 14)
    # plt.plot()



#  with fits.open(filename) as hdul:
#             # Events dataframe
#             events = hdul["Events"].data
#             events_table = Table(events)
#             events_cols = events.columns.names
#             df_events = pd.DataFrame.from_records(events_table, columns=events_cols)
#             df_events = df_events.sort_values(by=["time"])
#             # GTI (Good Time Interval) dataframe
#             gti = hdul["GTI"].data
#             gti_table = Table(gti)
#             gti_cols = gti.columns.names
#             df_gti = pd.DataFrame.from_records(gti_table, columns=gti_cols)
#             # Apply GTI filter to events
#             gti_mask = np.zeros(len(df_events), dtype=bool)
#             for i in range(len(df_gti)):
#                 start = df_gti.iloc[i]['START']
#                 stop = df_gti.iloc[i]['STOP']
#                 gti_mask |= (df_events["time"] >= start) & (df_events["time"] < stop)
#             df_events = df_events[gti_mask]
#             # Apply energy, pha, grade filters to events
#             df_events = df_events[(df_events['pha']>40) & (df_events['grade']>=0) & (df_events['energy']>500) & (df_events['energy']<7000)]
#             # Add obsid and region_id column (from filename)
#             try: 
#                 df_events["obsid"] = int(filename.split('_')[-5][-5:])
#             except: 
#                 df_events["obsid"] = int(filename.split('_')[-6][-5:])
#             df_events["region_id"] = int(filename.split('_')[-3][-4:])
#             # Append to dataframe list
#             list_df_events.append(df_events)
#             counter = counter+1
#             clear_output(wait=True)
#             print(f'Progress: {counter}/{total}')

#  region_filename = [region for region in glob.iglob(f'{global_path}/{set_id}/acisf*reg3.fits.gz') if str(obsid) in region and str(regionid) in region][0]
#         filtered_filename = event_filename.replace(".fits", "_filtered.fits")
#         print('Event Filename: ', event_filename)
#         print('Region Filename: ', region_filename)
#         filtered_filename = event_filename.replace(".fits", "_filtered.fits")
#         try:
#             ciao_contrib.runtool.dmcopy(f'{event_filename}[sky=region({region_filename})]', filtered_filename)
#             print('Filtered Event Filename: ', filtered_filename)
#         except OSError: 
#             print(f'{filtered_filename} already exists!')
#         counter = counter+1
#         clear_output(wait=True)
#         print(f'Progress: {counter}/{total}')
#['time', 'ccd_id', 'node_id', 'expno', 'chipx', 'chipy', 'tdetx', 'tdety', 'detx', 'dety', 'x', 'y', 'pha', 'pha_ro', 'energy', 'pi', 'fltgrade', 'grade', 'status']


    # #getting events (time+energy) from src_region
    # dmlist.punlearn()
    # dmlist.infile = f'{evt}[sky=region({src_region})][cols time, energy]'
    # dmlist.opt = 'data,clean'
    # out = dmlist().split('\n')
    # out_np = np.genfromtxt(out,dtype='str').astype('float64')
    # start_time = out_np[0,0]
    # src_times = out_np[::,0] - start_time
    # src_energies = out_np[::,1]


/Users/steven/Desktop/CSCData/23022need/acisf23022_000N027_r0122_regevt6_background_background.fits
/Users/steven/Desktop/CSCData/23022need/acisf23022_000N020_r0122_reg6.fits
/Users/steven/Desktop/CSCData/23022need/acisf23022_000N020_r0122_bkg6.fits
2881.232771688578
95312.0
33.08028457004581


In [ ]:

    
def sig_make_behr_file(BEHR_DIR,soft_src,hard_src,soft_bkg,hard_bkg,area,outfile,BEHR_outdir):
    with open(outfile,'w') as writeto:
        writeto.write(f'cd {BEHR_DIR}')
        writeto.write(f'\n echo "softsrc={soft_src} hardsrc={hard_src} softbkg={soft_bkg} hardbkg={hard_bkg} softarea={area} outputPr=True algo=quad"')
        writeto.write(f'\n./BEHR softsrc={soft_src} hardsrc={hard_src} softbkg={soft_bkg} hardbkg={hard_bkg} softarea={area} outputPr=true algo=quad output={BEHR_outdir}/BEHRresults')
    
    
    print('Running quad BEHR...')

    sig_run_BEHR(outfile)
    
    print("Reading BEHR...")
    
    data = np.loadtxt(f'{BEHR_outdir}/BEHRresults_prob.txt')
    c_value = data[::,4]
    prob = data[::,5]
    # plt.scatter(c_value,prob)
    # plt.show()
    
    return list(zip(c_value, prob))

def sig_make_behr_file_gibbs(BEHR_DIR,soft_src,hard_src,soft_bkg,hard_bkg,area,outfile,BEHR_outdir):
    with open(outfile,'w') as writeto:
        writeto.write(f'cd {BEHR_DIR}')
        writeto.write(f'\n echo "softsrc={soft_src} hardsrc={hard_src} softbkg={soft_bkg} hardbkg={hard_bkg} softarea={area} nsim=40000 algo=gibbs outputMC=True"')
        writeto.write(f'\n./BEHR softsrc={soft_src} hardsrc={hard_src} softbkg={soft_bkg} hardbkg={hard_bkg} softarea={area} nsim=40000 algo=gibbs outputMC=true output={BEHR_outdir}/BEHRresults')

    print('Running gibbs BEHR...')

    sig_run_BEHR(outfile)
        
    print("Reading BEHR...")
    
    data = np.loadtxt(f'{BEHR_outdir}/BEHRresults_draws.txt')
    softs = data[::,0]
    hards = data[::,1]
    vals = []
    
    for i in range(0,len(hards)):
        val = np.log10(softs[i]/hards[i])
        vals.append(val)    
    
    sample_mean = np.mean(vals)
    sample_std = np.std(vals)
    low = min(vals)
    high = max(vals)
    print('Mean=%.3f, Standard Deviation=%.3f' % (sample_mean, sample_std))
    dist = norm(sample_mean, sample_std)
    x = np.linspace(low,high,100)
    probabilities = [dist.pdf(value) for value in x]
    # plt.hist(vals, bins=100, density=True)
    # plt.plot(x, probabilities)
    # plt.show()    
    # curve_area = trapz(probabilities, dx=.01)
    # print("area =", curve_area)
    tot = sum(probabilities)
    probabilities = [x/tot for x in probabilities]

    return list(zip(x,probabilities))